# Parte 2: Clasificación con ANN 

Observamos overfitting en la gráfica. Además:
    
    loss = 0.01
    accuracy = 0.999 
    val_loss = 2.09
    val_accuracy = 0.68
    
Por tanto: 

    Etrain = 1-0.999 = 0.001
    Etest = 1- 0.68 = 0.32
    
    Bias = Etrain - Ehuman = 0.001
    Variance = Etest - Etrain = 0.32 - 0.001 = 0.319

Observamos que hay un bajo bias pero una alta varianza: podemos regularizar, cambiar la arquitectura, añadir más datos...

Se ha probado con , kernel_regularizer = keras.regularizers.l1_l2(0.001) pero ha tenido un mal resultado.
Se ha añadido Dropout(rate = rate_dropout[i]) rate_dropout = [0.8, 0.4, 0.2, 0.1]
Así: 

    loss = 0.345817
    accuracy = 0.868716 
    val_loss = 0.85344
    val_accuracy = 0.688326
    
Por tanto: 

    Etrain = 1 - 0.868716  = 0.13
    Etest = 1 - 0.688326 = 0.31
    
    Bias = Etrain - Ehuman = 0.13
    Variance = Etest - Etrain = 0.31 - 0.13 = 0.18
    
El bias vuelve a estar alto (13%)
			

### 1 - Import Libraries

In [1]:
# Tensorflow and tf.keras
import tensorflow as tf
from tensorflow import keras

#Helper libraries
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np


### 2 - Load data

In [2]:
class ROutput:
    def __init__(self, task, data):
        self.task = task
        self.data = data
        
class OutTaskData: 
    def __init__(self, task, data): 
        self.task = task
        self.data = data

In [3]:
import scipy.io as sio
# Primero leemos los registros
def read_outputs(rec):
    '''read_outputs("userS0091f1.mat")'''
    mat = sio.loadmat(rec)
    mdata = mat['session']
    val = mdata[0,0]
    #output = ROutput(np.array(val["task"]), np.array(val["data"]))
    output = ROutput(np.array(val["task"]), np.array(val["data"]))
    return output

In [5]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import Perceptron

total_records = 4 # CAMBIAR SI HAY MAS REGISTROS

task1 = 122 # SE PUEDE CAMBIAR
task2 = 123 # SE PUEDE CAMBIAR
task3 = 127 # SE PUEDE CAMBIAR
user = '0091' # SE PUEDE CAMBIAR

lTaskData = []
for i_rec in range(1,total_records+1):
        record = "./RegistrosDeAzorin/user#"+user+"#20040101#0"+str(i_rec)+"#reg001.mat"
        output = read_outputs(record) # output.task será y, output.data será x
        
        if task3 != "":
            outT = (output.task == task1) | (output.task == task2) | (output.task == task3) 
        else: 
            outT = (output.task == task1) | (output.task == task2)
      
        
        outData = output.data[0:np.shape(output.data)[0],outT[0,:]]
        outData = output.data[0:np.shape(output.data)[0],outT[0,:]]
        
        outTask = output.task[:,outT[0,:]]
        outTD = OutTaskData(outTask, outData)
        outTD.task = np.transpose(outTD.task)
        outTD.data = np.transpose(outTD.data)
        print(np.shape(outTD.task), np.shape(outTD.data))
        lTaskData.append(outTD)
        
# Vamos a coger 2 registros para el entrenamiento, 1 para el conjunto dev set, 1 para el test set
X_train, y_train, X_dev, y_dev = [],[],[],[] 
for j in range(0,total_records-2): # Cogemos 2 registros para entrenamiento
    X_train.extend(lTaskData[j].data)
    y_train.extend(lTaskData[j].task)


X_dev.extend(lTaskData[total_records-2].data) # Cogemos un registro para el dev set
y_dev.extend(lTaskData[total_records-2].task)

X_train = np.array(X_train)
y_train = np.ravel(np.array(y_train))
X_dev = np.array(X_dev)
y_dev = np.ravel(np.array(y_dev))

print ("X_train:",X_train.shape)
print ("y_train:",y_train.shape)

print ("X_dev:",X_dev.shape)
print ("y_dev:",y_dev.shape)

# ONE HOT ENCODER
encoder = make_pipeline(StandardScaler(), OneHotEncoder(categories="auto", sparse=False)) # Function that one-hot encodes integers))
y_one_hot = np.concatenate((y_train, y_dev), axis=0)
y_one_hot = encoder.fit_transform (y_one_hot.reshape(-1,1))
y_train = y_one_hot[:np.shape(y_train)[0]]
y_dev = y_one_hot[np.shape(y_train)[0]:]

print("ONE HOT ENCODER:")
print ("X_train:",X_train.shape)
print ("y_train:",y_train.shape)

print ("X_dev:",X_dev.shape)
print ("y_dev:",y_dev.shape)


X_test, y_test = [],[]
X_test.extend(lTaskData[total_records-1].data) # Cogemos un registro para el test set
y_test.extend(lTaskData[total_records-1].task)


X_test = np.array(X_test)
y_test = np.ravel(np.array(y_test))
print()
print ("X_test:",X_test.shape)
print ("y_test:",y_test.shape)

# ONE HOT ENCODER
encoder = make_pipeline(StandardScaler(), OneHotEncoder(categories="auto", sparse=False)) # Function that one-hot encodes integers))
y_test = encoder.fit_transform (y_test.reshape(-1,1)) # y_one_hot

print("ONE HOT ENCODER:")
print ("X_test:",X_test.shape)
print ("y_test:",y_test.shape)
    

(122368, 1) (122368, 32)
(121856, 1) (121856, 32)
(122368, 1) (122368, 32)
(120832, 1) (120832, 32)
X_train: (244224, 32)
y_train: (244224,)
X_dev: (122368, 32)
y_dev: (122368,)
ONE HOT ENCODER:
X_train: (244224, 32)
y_train: (244224, 3)
X_dev: (122368, 32)
y_dev: (122368, 3)

X_test: (120832, 32)
y_test: (120832,)
ONE HOT ENCODER:
X_test: (120832, 32)
y_test: (120832, 3)


### 3. Initialize variables

In [6]:
INPUTS = X_train.shape[1]
OUTPUTS = y_train.shape[1]
NUM_TRAINING_EXAMPLES = int(round(X_train.shape[0]/1))
NUM_DEV_EXAMPLES = int(round(y_train.shape[0]/1))

Some data is displayed to test correctness:

In [7]:
X_train[:2]

array([[  2376.6362 ,  -3993.0707 ,   -292.01509,  -1714.575  ,
         -8560.3748 , -12959.398  , -15365.081  ,   6104.5356 ,
          3067.4475 ,   4910.1941 ,   -543.51462,   -660.07691,
          3463.7905 ,  -4059.2894 ,  11748.775  ,   9284.2172 ,
          3479.9779 ,  -2588.5108 ,   3064.4787 ,  -3694.9775 ,
         -1908.1059 ,   4332.8826 , -13441.991  ,  -7562.5642 ,
         -4529.9448 ,  -3280.6658 ,  -3277.9158 ,   2412.3237 ,
         -2788.7917 ,   5308.7558 ,   7220.2523 ,  11014.589  ],
       [  2385.5737 ,  -3979.3833 ,   -284.7026 ,  -1706.6062 ,
         -8550.4998 , -12953.335  , -15362.925  ,   6113.7543 ,
          3079.7912 ,   4918.819  ,   -531.70214,   -650.48317,
          3474.6342 ,  -4048.6019 ,  11762.463  ,   9295.5297 ,
          3491.3842 ,  -2576.7296 ,   3075.9787 ,  -3683.0401 ,
         -1896.6059 ,   4344.7888 , -13428.397  ,  -7562.2517 ,
         -4517.101  ,  -3269.2596 ,  -3267.8221 ,   2422.1049 ,
         -2779.4792 ,   5306.6933 ,   7

In [8]:
X_dev[:2]

array([[  6107.5043 ,  -5426.6931 ,   -518.45217,    171.85906,
         -7640.2203 , -11927.744  , -11750.088  ,   4710.3819 ,
          5837.4736 ,   2545.0734 ,    693.73309,    293.98383,
           395.38989,  -3308.9158 ,  10355.028  ,   8593.156  ,
          4236.5078 ,   2777.2917 ,   6293.0352 ,  -1344.2944 ,
          1547.2628 ,   8932.1241 ,  -8558.7811 ,  -9692.154  ,
          -624.98322,  -2038.1994 ,   1662.9813 ,   1707.7312 ,
           266.51513,   6810.1593 ,   7133.4087 ,  12385.899  ],
       [  6117.7231 ,  -5417.9431 ,   -507.42094,    181.39029,
         -7630.6265 , -11920.275  , -11741.119  ,   4720.0069 ,
          5850.2861 ,   2556.1672 ,    707.57682,    305.98381,
           407.10862,  -3294.1658 ,  10369.621  ,   8603.781  ,
          4245.6015 ,   2785.198  ,   6303.3477 ,  -1341.4506 ,
          1555.5752 ,   8944.6553 ,  -8547.1873 ,  -9680.4977 ,
          -614.45199,  -2026.1056 ,   1674.1688 ,   1720.7937 ,
           278.10886,   6820.628  ,   7

In [9]:
y_train[:2]

array([[0., 0., 1.],
       [0., 0., 1.]])

In [10]:
y_dev[:2]

array([[0., 1., 0.],
       [0., 1., 0.]])

### 4 - Set hyperparameters

In [21]:
n_epochs = 2000
learning_rate = 0.001
batch_size = 250 
n_neurons_per_hlayer = [500, 250, 75, 25] # Number of units per layer, 4 hidden layers
rate_dropout = [0.8, 0.4, 0.2, 0.1]

### 5 - Build the model: 96-500-250-75-25-3 deep neural network architecture

In [22]:
model = keras.Sequential(name="DeepFeedforward")

model.add(keras.layers.InputLayer(input_shape=(INPUTS,), batch_size=None))

i = 0
for neurons in n_neurons_per_hlayer:
    model.add(keras.layers.Dense(neurons, activation="relu", kernel_initializer="he_normal"))
    #model.add(keras.layers.Dropout(rate = rate_dropout[i]))
    #model.add(keras.layers.BatchNormalization())
    i+=1

model.add(keras.layers.Dense(OUTPUTS, activation="softmax"))
model.summary()

Model: "DeepFeedforward"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 500)               16500     
_________________________________________________________________
dense_6 (Dense)              (None, 250)               125250    
_________________________________________________________________
dense_7 (Dense)              (None, 75)                18825     
_________________________________________________________________
dense_8 (Dense)              (None, 25)                1900      
_________________________________________________________________
dense_9 (Dense)              (None, 3)                 78        
Total params: 162,553
Trainable params: 162,553
Non-trainable params: 0
_________________________________________________________________


In [23]:
model.layers

In [24]:
for l in model.layers: print (l.name)

dense_5
dense_6
dense_7
dense_8
dense_9


All the parameters of a layer can bee accessed:

In [25]:
weights, biases = model.layers[0].get_weights()
weights.shape

(32, 500)

In [26]:
weights

array([[ 0.24012922,  0.5682347 , -0.2092413 , ...,  0.03145886,
         0.36078095, -0.28126195],
       [-0.04698908,  0.26469404,  0.43809462, ..., -0.56633115,
        -0.21256223,  0.49595243],
       [ 0.03572486, -0.09099077,  0.13319801, ..., -0.07594829,
         0.10134641,  0.24191952],
       ...,
       [ 0.41282037,  0.00662193, -0.39523727, ..., -0.0466457 ,
         0.02559994, -0.39150625],
       [-0.42238045, -0.49491793, -0.2566481 , ...,  0.25114343,
        -0.1365795 ,  0.21539097],
       [ 0.22570309,  0.11395899,  0.02570103, ..., -0.18149869,
        -0.16666283,  0.38738167]], dtype=float32)

In [27]:
biases # Bias de la primera capa: 500 bias inicializados a 0 

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [28]:
biases.shape

(500,)

### 6. Compile the model

In [29]:
model.compile(loss=tf.keras.losses.categorical_crossentropy,
              optimizer=tf.keras.optimizers.Adam(lr=learning_rate, beta_1=0.9, beta_2=0.999),
              metrics=["categorical_accuracy"])


### 7. Train the model with M-BGD

In [ ]:
import time
start = time.time()
history = model.fit(X_train, y_train, batch_size=batch_size, epochs=n_epochs, verbose=1, validation_data=(X_dev, y_dev))    
print (time.time() - start)

Epoch 1/2000
977/977 [==============================] - 16s 15ms/step - loss: 538.7100 - categorical_accuracy: 0.3528 - val_loss: 160.6158 - val_categorical_accuracy: 0.3975
Epoch 2/2000
977/977 [==============================] - 13s 13ms/step - loss: 55.0120 - categorical_accuracy: 0.3966 - val_loss: 125.2513 - val_categorical_accuracy: 0.2803
Epoch 3/2000
977/977 [==============================] - 13s 14ms/step - loss: 11.6136 - categorical_accuracy: 0.4398 - val_loss: 36.5344 - val_categorical_accuracy: 0.3973
Epoch 4/2000
977/977 [==============================] - 13s 13ms/step - loss: 2.6617 - categorical_accuracy: 0.5303 - val_loss: 42.8657 - val_categorical_accuracy: 0.2904
Epoch 5/2000
977/977 [==============================] - 13s 14ms/step - loss: 1.4077 - categorical_accuracy: 0.5994 - val_loss: 44.5819 - val_categorical_accuracy: 0.3052
Epoch 6/2000
977/977 [==============================] - 13s 13ms/step - loss: 4.2840 - categorical_accuracy: 0.4184 - val_loss: 39.9398 - v

977/977 [==============================] - 10s 11ms/step - loss: 1.0849 - categorical_accuracy: 0.3868 - val_loss: 170.8678 - val_categorical_accuracy: 0.3975
Epoch 96/2000
977/977 [==============================] - 10s 10ms/step - loss: 1.0846 - categorical_accuracy: 0.3890 - val_loss: 170.9116 - val_categorical_accuracy: 0.3975
Epoch 97/2000
977/977 [==============================] - 11s 11ms/step - loss: 1.0844 - categorical_accuracy: 0.3878 - val_loss: 168.2817 - val_categorical_accuracy: 0.3975
Epoch 98/2000
977/977 [==============================] - 11s 11ms/step - loss: 1.0844 - categorical_accuracy: 0.3878 - val_loss: 168.8708 - val_categorical_accuracy: 0.3975
Epoch 99/2000
977/977 [==============================] - 10s 10ms/step - loss: 1.0848 - categorical_accuracy: 0.3865 - val_loss: 167.2402 - val_categorical_accuracy: 0.3975
Epoch 100/2000
977/977 [==============================] - 11s 11ms/step - loss: 1.0841 - categorical_accuracy: 0.3880 - val_loss: 166.3116 - val_cate

977/977 [==============================] - 10s 10ms/step - loss: 1.0838 - categorical_accuracy: 0.3893 - val_loss: 94.4336 - val_categorical_accuracy: 0.3889
Epoch 190/2000
977/977 [==============================] - 10s 10ms/step - loss: 1.0845 - categorical_accuracy: 0.3876 - val_loss: 94.3458 - val_categorical_accuracy: 0.3886
Epoch 191/2000
977/977 [==============================] - 11s 11ms/step - loss: 1.0849 - categorical_accuracy: 0.3866 - val_loss: 94.3480 - val_categorical_accuracy: 0.3886
Epoch 192/2000
977/977 [==============================] - 12s 12ms/step - loss: 1.0844 - categorical_accuracy: 0.3877 - val_loss: 94.3486 - val_categorical_accuracy: 0.3886
Epoch 193/2000
977/977 [==============================] - 13s 13ms/step - loss: 1.0843 - categorical_accuracy: 0.3878 - val_loss: 94.3550 - val_categorical_accuracy: 0.3886
Epoch 194/2000
977/977 [==============================] - 13s 14ms/step - loss: 1.0841 - categorical_accuracy: 0.3880 - val_loss: 94.3513 - val_catego

977/977 [==============================] - 10s 11ms/step - loss: 1.0844 - categorical_accuracy: 0.3860 - val_loss: 94.3540 - val_categorical_accuracy: 0.3886
Epoch 284/2000
977/977 [==============================] - 11s 11ms/step - loss: 1.0848 - categorical_accuracy: 0.3880 - val_loss: 94.3525 - val_categorical_accuracy: 0.3886
Epoch 285/2000
977/977 [==============================] - 11s 11ms/step - loss: 1.0838 - categorical_accuracy: 0.3889 - val_loss: 94.3429 - val_categorical_accuracy: 0.3886
Epoch 286/2000
977/977 [==============================] - 10s 10ms/step - loss: 1.0843 - categorical_accuracy: 0.3883 - val_loss: 94.3469 - val_categorical_accuracy: 0.3886
Epoch 287/2000
977/977 [==============================] - 10s 10ms/step - loss: 1.0852 - categorical_accuracy: 0.3870 - val_loss: 94.3430 - val_categorical_accuracy: 0.3886
Epoch 288/2000
977/977 [==============================] - 10s 10ms/step - loss: 1.0846 - categorical_accuracy: 0.3875 - val_loss: 94.3467 - val_catego

977/977 [==============================] - 10s 10ms/step - loss: 1.0844 - categorical_accuracy: 0.3876 - val_loss: 94.3425 - val_categorical_accuracy: 0.3886
Epoch 378/2000
977/977 [==============================] - 10s 10ms/step - loss: 1.0846 - categorical_accuracy: 0.3883 - val_loss: 94.3514 - val_categorical_accuracy: 0.3886
Epoch 379/2000
977/977 [==============================] - 10s 10ms/step - loss: 1.0845 - categorical_accuracy: 0.3870 - val_loss: 94.3533 - val_categorical_accuracy: 0.3886
Epoch 380/2000
977/977 [==============================] - 10s 10ms/step - loss: 1.0853 - categorical_accuracy: 0.3861 - val_loss: 94.3488 - val_categorical_accuracy: 0.3886
Epoch 381/2000
977/977 [==============================] - 10s 10ms/step - loss: 1.0847 - categorical_accuracy: 0.3867 - val_loss: 94.3574 - val_categorical_accuracy: 0.3886
Epoch 382/2000
977/977 [==============================] - 14s 14ms/step - loss: 1.0845 - categorical_accuracy: 0.3878 - val_loss: 94.3511 - val_catego

977/977 [==============================] - 10s 10ms/step - loss: 1.0842 - categorical_accuracy: 0.3888 - val_loss: 94.3430 - val_categorical_accuracy: 0.3886
Epoch 472/2000
977/977 [==============================] - 10s 10ms/step - loss: 1.0846 - categorical_accuracy: 0.3881 - val_loss: 94.3459 - val_categorical_accuracy: 0.3886
Epoch 473/2000
977/977 [==============================] - 10s 10ms/step - loss: 1.0854 - categorical_accuracy: 0.3859 - val_loss: 94.3518 - val_categorical_accuracy: 0.3886
Epoch 474/2000
977/977 [==============================] - 10s 10ms/step - loss: 1.0845 - categorical_accuracy: 0.3897 - val_loss: 94.3481 - val_categorical_accuracy: 0.3886
Epoch 475/2000
977/977 [==============================] - 10s 10ms/step - loss: 1.0848 - categorical_accuracy: 0.3864 - val_loss: 94.3574 - val_categorical_accuracy: 0.3886
Epoch 476/2000
977/977 [==============================] - 10s 10ms/step - loss: 1.0841 - categorical_accuracy: 0.3877 - val_loss: 94.3507 - val_catego

### 8. Get the results

In [ ]:
results=pd.DataFrame(history.history)
results.plot(figsize=(8, 5))
plt.grid(True)
plt.xlabel ("Epochs")
plt.ylabel ("Accuracy - Mean Log Loss")
plt.gca().set_ylim(0, 2) # set the vertical range to [0-1]
plt.show()

# categorical_accuracy es el training accuracy
# val_categorical_accuracy es el validation accuracy
# loss es el training loss
# val_loss es el validation loss

# Overfitting:
# Por un lado la Accuracy de los datos de entrenamiento aumenta con las epochs,
# mientras que la Accuracy de los datos de validación disminuye o se mantiene constante a lo largo de las epochs. 
# La Loss de los datos de validación alcanza su mínimo después de pocos epochs y luego empieza a subir, 
# mientras que la Loss de los datos de entrenamiento disminuye linealmente.

In [ ]:
history.params

In [ ]:
results[-1:]

In [ ]:
print ("Accuracy for the training set: ", results.categorical_accuracy.values[-1:][0])

In [ ]:
print ("Accuracy for the development test set: ", results.val_categorical_accuracy.values[-1:][0])

#### Let's see how the model predicts using the development test set:

In [ ]:
dev_predictions=model.predict(X_dev).round(2) 
dev_predictions[:20]

In [ ]:
dev_rounded_predictions=np.round(dev_predictions)
indices = np.argmax(dev_predictions,1)
for row, index in zip(dev_rounded_predictions, indices): row[index]=1
dev_rounded_predictions[:20]

In [ ]:
y_dev[:20]

In [ ]:
dev_correct_predictions = np.equal(np.argmax(dev_rounded_predictions,1),np.argmax(y_dev,1))
print (dev_correct_predictions[:30])

In [ ]:
from collections import Counter
Counter (dev_correct_predictions)

### 9. Save model

We save the model if it is better than others 

In [ ]:
# Save
model.save('./Modelos/EGG_prac1_ANN_model3.h5') # Change the name in each model

In [ ]:
# Extract
"""
new_model = keras.models.load_model('./Modelos/EGG_prac1_ANN_model3.h5')

import numpy as np

# Verify state
new_predictions = new_model.predict(X_dev)
np.testing.assert_allclose(predictions, new_predictions, rtol=1e-6, atol=1e-6)

# Note that the optimizer state is also preserved:
# you can resume training where you left off.
"""

### 10 - Final Test

#### 10.1 - Load the final test set

In [ ]:

"""
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import Perceptron

total_records = 4 # CAMBIAR SI HAY MAS REGISTROS

task1 = 122 # SE PUEDE CAMBIAR
task2 = 123 # SE PUEDE CAMBIAR
task3 = 127 # SE PUEDE CAMBIAR
users = ["0091"] # SE PUEDE CAMBIAR
"""

In [ ]:
"""
def readRegisterAndReturnXy(user, total_records, task1, task2, task3 = ""):
    lTaskData = []
    for i_rec in range(1,total_records+1):
            record = "userS"+user+"f"+str(i_rec)+".mat"
            output = read_outputs(record) # output.task será y, output.data será x

            if task3 != "":
                outT = (output.task == task1) | (output.task == task2) | (output.task == task3) 
            else: 
                outT = (output.task == task1) | (output.task == task2)
            outData = output.data[outT[:,0],0:np.shape(output.data)[1]]

            outTask = output.task[outT[:,0]]
            outTD = OutTaskData(outTask, outData)
            lTaskData.append(outTD)

    X_test, y_test = [],[]
    for j in range(0,total_records):
        X_test.extend(lTaskData[j].data)
        y_test.extend(lTaskData[j].task)

    X_test = np.array(X_test)
    y_test = np.ravel(np.array(y_test))

    print ("X_test:",X_test.shape)
    print ("y_test:",y_test.shape)

    # ONE HOT ENCODER
    encoder = make_pipeline(StandardScaler(), OneHotEncoder(categories="auto", sparse=False)) # Function that one-hot encodes integers))
    y_test = encoder.fit_transform (y_test.reshape(-1,1)) # y_one_hot

    print("ONE HOT ENCODER:")
    print ("X_test:",X_test.shape)
    print ("y_test:",y_test.shape)
    
    return X_test, y_test
"""

In [ ]:
"""
n_final_test = n_instances-n_train-n_dev

x_final_test = attributes.values[n_train+n_dev:n_instances]
t_final_test = label.values[n_train+n_dev:n_instances]

print ("x_test:",x_final_test.shape)
print ("t_test:",t_final_test.shape)


"""

#### 10.2 - Evaluate the model

In [ ]:
from collections import Counter
def predictWithModel(model, X_test, y_test, task1, task2, task3=""):
    model.evaluate(X_test, y_test)
    test_predictions = model.predict(X_test)
    test_rounded_predictions=np.round(test_predictions)
    indices = np.argmax(test_predictions,1)
    for row, index in zip(test_rounded_predictions, indices): row[index]=1
    print(test_rounded_predictions[:20])
    
    # ACCURACY:
    test_correct_predictions = np.equal(np.argmax(test_rounded_predictions,1),np.argmax(y_test,1))
    print()
    print(test_correct_predictions)
    print(type(test_correct_predictions))
    final_test_prediction_results=Counter(test_correct_predictions)
    
    success = np.mean(test_rounded_predictions == y_test)*100
    
    return final_test_prediction_results, success

In [ ]:
users = ["0091"]
for u in users:   
    print("USER:", u)
    #X_test, y_test = readRegisterAndReturnXy(u, total_records, 122, 123, 127)
    accuracy, success = predictWithModel(model, X_test, y_test, 122, 123, 127)
    print(u, accuracy, success)
    print()

In [ ]:
#model.evaluate(x_final_test, t_final_test) # Un 77 es un accuracy bajo 

The outputs computed by the neural network for the final testing dataset.

In [ ]:
"""
test_predictions=model.predict(x_final_test)
test_rounded_predictions=np.round(test_predictions)
indices = np.argmax(test_predictions,1)
for row, index in zip(test_rounded_predictions, indices): row[index]=1
test_rounded_predictions[:20]
"""

The target outputs.

In [ ]:
# t_final_test[:20]

The first 30 predictions. True means that the neural network correctly classifies the input vector.

In [ ]:
# test_correct_predictions = np.equal(np.argmax(test_rounded_predictions,1),np.argmax(t_final_test,1))
# test_correct_predictions[:30]

#### The final test accuracy.

In [ ]:
"""from collections import Counter
final_test_prediction_results=Counter(test_correct_predictions)
final_test_prediction_results"""

In [ ]:
#final_test_prediction_results[True]/sum(final_test_prediction_results.values())